In [11]:
import sys, os, time, datetime    # 서버와 통신할 때 중간중간 시간 지연. 보통은 1초
import pandas as pd  # 판다스 : 데이터분석 라이브러리
import numpy as np   # 넘파이 : 숫자, 행렬 데이터 라이브러리
import requests
from urllib.request import urlopen
import urllib

from bs4 import BeautifulSoup

from selenium import webdriver   # 웹 브라우저 자동화
from selenium.webdriver import ActionChains
from selenium.webdriver.common.actions.action_builder import ActionBuilder
from selenium.webdriver.common.actions.mouse_button import MouseButton
from selenium.webdriver.common.by import By

import FinanceDataReader as fdr

# path = 'C:/Users/Admin/py_stock_analysis/'
path = 'C:/Users/kohyo/Documents/py_stock_analysis/py_stock_analysis/'

In [2]:
#크롬 웹브라우저 실행
driver = webdriver.Chrome()
url = "https://finance.naver.com/sise/theme.nhn?&page=1"
driver.get(url)
time.sleep(2)

In [ ]:
# 7페이지 for문으로 반복하여 테마명 데이타 
for i in range(7):
    html = requests.get('https://finance.naver.com/sise/theme.nhn?&page={}'.format(i + 1))
    table = pd.read_html(html.text)
    if i == 0:
        stock = table[0].dropna().reset_index(drop = True)
    else:
        stock1 = table[0].dropna().reset_index(drop = True)
        stock = pd.concat([stock, stock1])

stock = stock.reset_index(drop = True)
stock.drop(stock.columns[1:], axis=1, inplace=True) ### 첫째열 제외하고 삭제
stock = stock.set_axis(['theme'], axis=1) ### 칼럼 이름 변경
pd.options.display.max_rows = 4000
print(stock)

In [ ]:
### 1페이지부터 7페이지까지 모두 이동하면서 모든 행을 클릭해서 테마이름별 No 정보 취함
def theme_url_load(page):
    url = ('https://finance.naver.com/sise/theme.nhn?&page={}'.format(page))
    driver.get(url)
    time.sleep(1)
k=1
l_data = []

first_url = "https://finance.naver.com/sise/theme.naver"
driver.get(first_url)
print('테마개수 : {0}'.format(len(stock)))

for i in range(1, 7):
    theme_url_load(i)
    for j in range(1,40):
        if k <= len(stock):
            print(i, j, k)
            quotient, remainder = divmod(j, 5)
            if remainder == 0:
                quotient-=1
            table_row = j + (quotient+1) * 3    
                        
            xpath = ('//*[@id="contentarea_left"]/table[1]/tbody/tr[{}]/td[1]/a'.format(table_row))
            print(table_row, xpath)
            
            clickable = driver.find_element(By.XPATH, xpath)
            ActionChains(driver)\
                .click(clickable)\
                .perform()
            
            url = driver.current_url ### 페이지의 URL을 받음           
            html = requests.get(url)
            # using the BeautifulSoup module
            soup = BeautifulSoup(html.text, 'html.parser')

            for title in soup.find_all('title'):
                str_title = title.get_text().strip()
            x = str_title.find(':')
            str_title = str_title[:x-1]
            print(str_title, url)
            l_data.append([str_title, url])
            k += 1
            driver.back()
            time.sleep(1)

print(l_data)  

path = 'C:/Users/Admin/py_stock_analysis/'

df = pd.DataFrame(l_data)
df.to_csv(path+'thema_url.csv', encoding='utf-8', index=False)

In [72]:
# 테마 csv 파일을 읽고 url애서 테마의 no를 추출
columns = ['theme', 'url']
df = pd.read_csv(path+'thema_url.csv', names=columns, encoding='utf-8', header=None, skiprows=1)

df['theme_no'] = df['url'].str[69:]
df.head()

,theme,url,theme_no
0,야놀자(Yanolja),https://finance.naver.com/sise/sise_group_deta...,483
1,인터넷 대표주,https://finance.naver.com/sise/sise_group_deta...,49
2,일자리(취업),https://finance.naver.com/sise/sise_group_deta...,324
3,가상화폐(비트코인 등),https://finance.naver.com/sise/sise_group_deta...,331
4,건설 대표주,https://finance.naver.com/sise/sise_group_deta...,154


In [ ]:
# 테마별 url에 들어가서 주식종목별 데이타 입수
l_data = []
for d in df.iterrows():
    theme = d[1][0]
    theme_no = d[1][2]
    url = urllib.parse.quote_plus(d[1][1], "/:?=&") # need this line
    driver.get(url)
    html = requests.get(url)
    # using the BeautifulSoup module
    soup = BeautifulSoup(html.text, 'html.parser')

    for row in soup.select('tbody tr'):
        row_text = [x.text for x in row.find_all('td')]
        if row_text[0] != '':
            if row_text[0][-1:] == '*':
                market = 'kosdaq'
            else: market = 'kospi'
            print(theme, theme_no, market, row_text[0])
            l_data.append([theme, theme_no, market, row_text[0]])
df = pd.DataFrame(l_data)
df.to_csv(path+'thema_stock.csv', encoding='utf-8', index=False)

In [12]:
### 테마 주식종목을 import
columns = ['theme', 'theme no', 'market', 'stock']
df_theme = pd.read_csv(path+'thema_stock.csv', names=columns, encoding='utf-8', header=None, skiprows=1)
# 종목명에서 *를 제거해야함
df_theme['stock'] = df_theme['stock'].str.replace(' *', '')
# 종목정보 데이타를 Import
df_stock = pd.read_excel(path+'stock_code.xlsx')
# df_stock에 df_theme를 map해서 테마명을 열에 추가
df_stock = df_stock.merge(df_theme, left_on='한글 종목명', right_on='stock')
df_stock.drop(columns=['market', 'stock'], inplace=True)
df_stock.head()

c:\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,표준코드,단축코드,한글 종목명,한글 종목약명,영문 종목명,상장일,시장구분,증권구분,소속부,주식종류,액면가,상장주식수,theme,theme no
0,KR7050120005,050120,ES큐브,ES큐브,ES CUBE,2002/04/25,KOSDAQ,주권,벤처기업부,보통주,5000,13564086,가상화폐(비트코인 등),331
1,KR7024850000,024850,HLB이노베이션,HLB이노베이션,HLB innoVation,2001/01/18,KOSDAQ,주권,중견기업부,보통주,500,66507231,반도체 재료/부품,14
2,KR7024120008,024120,KB오토시스,KB오토시스,"KB Autosys Co.,Ltd",1994/12/07,KOSDAQ,주권,중견기업부,보통주,500,11500000,자동차부품,27
3,KR7044180008,044180,KD,KD,KD Corporation,2000/11/16,KOSDAQ,주권,투자주의환기종목(소속부없음),보통주,500,26717799,건설 중소형,322
4,KR7086960002,086960,MDS테크,MDS테크,MDS Tech,2006/09/26,KOSDAQ,주권,중견기업부,보통주,200,87578536,NFT(대체불가토큰),492


In [16]:
# df_stock을 한 행씩 돌면서 각 종목의 과거 20년치 주가데이터 및 공시자료를 다운로드해서 각각 파일로 저장
# target = '삼성전자'
# start = '20100101'
# end = '20101231'

# symbols = pd.read_csv('./krx_stock_symbols.csv')
# code = symbols[symbols['Name'] == target]['Code'].iloc[0]

# stock_data = fdr.DataReader(code, start, end)

# stock_data.to_csv('./stock_price.csv')
for idx, stock in df_stock.iterrows():
    if idx < 5:
        stock[idx][0]
    else: break

C:\Users\kohyo\AppData\Local\Temp\ipykernel_5224\2568542673.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  stock[idx][0]


In [8]:
# [기본분석 1] PBR, PER, ROE, EPS 산출

In [9]:
# [기본분석 2] PBR, PER, ROE, EPS의 성장율과 주가수익 간의 상관관계 산출, 지표별 연간실적대비, 차년의 연초 주가대비 연말 주가 성장율로 상관관계 산출
# 종목코드, 종목명, 연도 2023, 연초주가, 연말주가, 주가성장율, PBR 전전년, PBR 전년, PBR성장율, PBR상관계수, PER 전전년, PER 전년, PER상관계수, ROE 전전년, ROE 전년, ROE 상관계수, EPS 전전년, EPS 전년, EPS 상관계수

In [ ]:
# [기본분석 3] 테마별 기본분석 1~2 수행, 테마내 최우수 3개 종목 탐색

In [10]:
# [차트 1] 종목별 주가 데이타를 활용해서 현재 정배열 상태인지 여부 학인 (20일 > 60일> 120일 > 200일)
# 정배열인 경우 일봉 차트 출력

In [7]:
# [차트 2] 종목별 주가 데이타를 활용해서 골든 크로스 발생 여부 확인 20일선, 60일선 120일선 200일선
# 골든크로스인 경우 일봉 차트 출력

In [ ]:
# [차트 3] 차트1, 차트2에 해당하는 기업의 거래량 증가여부 확인

In [11]:
# [차트 4] 차트1~4 기업의 RSI 과열여부 확인

In [ ]:
# [차트 4] 테마별, 종목별 분기별 주가, 거래량 증가율 - 시장관심 종목 탐색 (시장평균 증가율대비 상위)

In [ ]:
# [차트 5] 거래량 급증 테마, 종목 확인, 과거 10년 평균대비
# 거래량 급증종목 차트 출력